# Sprint_03 機械学習スクラッチ　線形回帰

In [1]:
class ScratchLinearRegression():
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ
    self.loss : 次の形のndarray, shape (self.iter,)
      訓練データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証データに対する損失の記録

    """
    def __init__(self, num_iter, lr, no_bias, verbose):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.no_bias = no_bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        
        _gradient_descent(self, X)
        
        pass

    def _gradient_descent(self, X, error):
        """
        fitメソッドで呼び出された際に最急降下法による学習を行う。

        Parameters
        ----------
        self: class関数を引用する。

        X : 次の形のndarray, shape (n_samples, n_features)

        error :

        """

        theta -= (sum(_linear_hypothesis(X, theta)-y)/m) * X

        pass
    
    def _linear_hypothesis(self, X, theta):
    """
    線形の仮定関数を計算する

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    
    theta : 次の形のndarray, shape (n_samples, )
    　パラメータ
    
    Returns
    -------
      次の形のndarray, shape (n_samples, 1)
      線形の仮定関数による推定結果
    
    """
    solution = theta[0] + np.sum(np.matmul(theta[1: ], X))
    
    return solution
    
    def predict(self, X):
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """
        pass
        return

## 【問題1】仮定関数

以下の数式で表される線形回帰の仮定関数を実装してください。

$$
h_\theta(x) = \theta_0x_0 + \theta_1x_1+...+\theta_jx_j+...+\theta_nx_n.(x_0=1)
$$

### 【目的】

* 上記の重回帰式を仮定する関数を作成する。

### 【考察】

* $\begin{align}\theta\end{align}$ は直線の傾き（coefficients）を制御している。
* $\begin{align}x_0 = 1\end{align}$ のため、切片は1である。
* 行列 $\begin{align}\theta\end{align}$ と行列 $\begin{align}x_n\end{align}$ の内積（np.dot or np.matmul）を算出する関数を作成する。
* 内積を算出するため、引数Xの行数分の配列数thetaをnp.randomを用いて算出する。

### 【工程順序】
1. np.random（0-10の範囲）を用いて、Xの行数分の配列数thetaのarrayを構築
2. x_train（ここでは引数X）を用いて$\begin{align}h_\theta\end{align}$(x)を求める。


In [1]:
# Importing libraries

import numpy as np

In [23]:
def _linear_hypothesis(self, X, theta):
    """
    線形の仮定関数を計算する

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    
    theta : 次の形のndarray, shape (n_samples, )
    　パラメータ
    
    Returns
    -------
      次の形のndarray, shape (n_samples, 1)
      線形の仮定関数による推定結果

    Process
    -------    
    'theta' is the coefficients.
    there are n independent variables.
    
    パラメータが２つしかない場合
    
    def hypothesis(theta, X):
        return theta[0] + theta[1]*X
    def cost_calc(theta, X, y):
        return (1/2*m) * np.sum((hypothesis(theta, X) - y)**2)
    
    m = len(X) # number_of_training_data
    def gradient_descent(theta, X, y, epoch, alpha):
        cost = []
        i = 0
        while i < epoch:
            hx = hypothesis(theta, X)
            theta[0] -= alpha * (sum(hx-y)/m)
    
    """
    
    solution = theta[0] + np.sum(np.matmul(theta[1: ], X))
    
    
    return solution

## 【問題2】最急降下法

最急降下法により学習させる実装を行なう。

以下の式で表されるパラメータの更新式のメソッド_gradient_descentを追加し、fitメソッドから呼び出すようにする。

$\begin{align}
\theta_j := \theta_j-\alpha\frac{1}{m}\sum_{i=1}^{m}[(h_\theta(x^{(i)})-y^{(i)})x_j^{(i)}]
\end{align}$

$\begin{align}\theta_j\end{align}$：

$\begin{align}\alpha\end{align}$：学習率

$\begin{align}i\end{align}$：サンプルのインデックス

$\begin{align}j\end{align}$：特徴量のインデックス

$\begin{align}m\end{align}$：サンプルの最大インデックス値（繰り返し回数の最大値）

$h_\theta(x^{(i)})$：_linear_hypothesis

$\begin{align}y^{(i)}\end{align}$：y_train

$\begin{align}x_j^{(i)}\end{align}$：X_train

### 【目的】

* ひな形_gradient_descent（勾配降下）を用いて、最急降下法による機械学習が行えるように関数を完成させる。

### 【考察】

* 最急降下法とは、接線の傾き（$\theta$）をゼロに近づくようにxの値を更新していき最適解に収束させる方法。
* := の意味は「左辺を右辺で定義（代入）する」
* 偏微分：特定の文字以外を定数とみなして微分したものを偏微分（偏導関数）と言います。
    * 微分：ある関数の任意の点における傾き（$\theta$）を導く式「導関数」を求めること。
    * 傾き（$\theta$）を求めるには2点間の変化の割合を求めること。
        * $変化の割合=\frac{yの増加量}{xの増加量}$
        

最急降下法のアルゴリズム
1. 対象とする関数を$(h_\theta(x^{(i)})$とし、関数$(h_\theta(x^{(i)})$の引数のベクトルをX
2. $h_\theta(x^{(i)})-y^{(i)}$が偏差となる。


In [3]:
def _gradient_descent(self, X, error):
    """
    fitメソッドで呼び出された際に最急降下法による学習を行う。

    Parameters
    ----------
    self: class関数を引用する。
    
    X : 次の形のndarray, shape (n_samples, n_features)
    
    error : 

    Returns
    -------
      次の形のndarray, shape (n_samples, 1)
      線形の仮定関数による推定結果    
        
    
    """

    # h_thetaとyはnumpy.ndarrayであること
    # inside_sigma = np.multiply((h_theta - y_train), X_train)
    
    # theta = theta - lr * (1/m) * np.sum(inside_sigma, axis=0)
    # or
    theta -= (sum(_linear_hypothesis(X, theta)-y)/m) * X
    
    pass

## 【問題3】推定